In [4]:
from afterpay_gdp_interfaces import RedshiftHook
import datetime
import json
from enum import Enum  
import pandas as pd
import json

class Metric(Enum):
    OrderCnt = 'num_of_order'
    AOV = 'aov'
    GMV = 'gmv'
    NewCustomer = 'new_cus'
    
class Period(Enum):
    Week = '7d'
    Month = '30d'
    Three_month = '90d'
    Year = '12m'
    
#change to your okta_username
r = RedshiftHook(cluster='vega', okta_username='zhou.fang@afterpay.com')
conn = r.get_conn() 

def get_ratio(current_series, prev_series):
    return 'na' if len(prev_series) == 0 \
                else (current_series.values[0] - prev_series.values[0])/prev_series.values[0] * 100

def get_prev_cob(cob_str, period=Period.Week):
    if period == Period.Month:
        return "dateadd(day, -30, '{}')".format(cob_str)
    elif period == Period.Three_month:
        return "dateadd(day, -90, '{}')".format(cob_str)
    elif period == Period.Year:
        return "dateadd(year, -1, '{}')".format(cob_str)
    return "dateadd(day, -7, '{}')".format(cob_str)
        
def get_summary(merchant_id, start_dt, period=Period.Week):
    merchant_df = r.get_pandas_df('''select cast(t.cob_date as varchar) as cob,
                                     t.* from sandbox_analytics_us.cdp_merchant_daily t
                                     where merchant_id={m_id} and (cob_date = '{start_dt}' or cob_date = {prev_dt})
                                  '''.format(m_id = merchant_id, start_dt = start_dt, prev_dt = get_prev_cob(start_dt, period))) 

    merchant_df = merchant_df[['cob', 'num_of_order_' + period.value, 'aov_' + period.value \
                               , 'gmv_' + period.value, 'new_cus_' + period.value]] \
                  .rename(columns = { \
                        'num_of_order_' + period.value: 'num_of_order', \
                        'aov_' + period.value: 'aov', \
                        'gmv_' + period.value: 'gmv', \
                        'new_cus_' + period.value: 'new_cus' })

    current = merchant_df[merchant_df['cob'] == start_dt]
    prev = merchant_df[merchant_df['cob'] != start_dt]

    final = {
                'order': 
                         {'amount': current['num_of_order'].values[0], 
                          'ratio': get_ratio(current['num_of_order'], prev['num_of_order'])} ,
                          
                'newCustomer': 
                         {'amount': current['new_cus'].values[0], 
                          'ratio': get_ratio(current['new_cus'], prev['new_cus'])} ,
        
                'averageOrderValue': 
                         {'amount': current['aov'].values[0], 
                          'ratio': get_ratio(current['aov'], prev['aov'])} ,
        
                'totalRevenue': 
                         {'amount': current['gmv'].values[0], 
                          'ratio': get_ratio(current['gmv'], prev['gmv'])}
            }
    
    return final #json.dumps(final)

In [5]:
get_summary(42267, '2021-03-19')

{'order': {'amount': 7934.0, 'ratio': 15.420424789060228},
 'newCustomer': {'amount': 2700.0, 'ratio': 13.445378151260504},
 'averageOrderValue': {'amount': 182.0728, 'ratio': -0.053521347881623174},
 'totalRevenue': {'amount': 1444566.22, 'ratio': 15.358684658559648}}